In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import xlsxwriter

In [2]:
# URLs to scrape
urls = ['https://www.youtube.com/@chess/videos']

In [19]:
# Initialize variables
times = 0
row = 0
t, v, d = [], [], []

service = Service(executable_path=r"C:\Program Files (x86)\Google\chromedriver-win64\chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Loop through each URL
for url in urls:
    driver.get(f'{url}/videos?view=0&sort=p&flow=grid')
    times = 0
    while times < 5:
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        times += 1
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content, 'html.parser')

# Extract video titles
title_elements = soup.findAll('yt-formatted-string', id='video-title')
for title in title_elements:
    t.append(title.text.strip()) 

# Extract views
view_elements = soup.findAll('span', class_='inline-metadata-item style-scope ytd-video-meta-block')
for view in view_elements:
    if 'просмотров' in view.text:
        v.append(view.text.strip().replace("\u00a0тыс.", "K").replace("\u00a0млн", "M"))

# Extract duration
duration_elements = soup.findAll('span', class_='style-scope ytd-thumbnail-overlay-time-status-renderer')
for duration in duration_elements:
        aria_label = duration.get('aria-label')
        if aria_label and any(char.isdigit() for char in aria_label):
            # Append only if this duration has not already been added
            if aria_label.strip() not in d:
                d.append(aria_label.strip())

driver.quit()


In [20]:
# Creating an Excel workbook and worksheet
workbook = xlsxwriter.Workbook('file.xlsx')
worksheet = workbook.add_worksheet()

# Writing headers
worksheet.write(0, 0, "Title")
worksheet.write(0, 1, "Views")
worksheet.write(0, 2, "Duration")

# Writing data to the worksheet
row = 1
for title, view, duration in zip(t, v, d):
    worksheet.write(row, 0, title)
    worksheet.write(row, 1, view)
    worksheet.write(row, 2, duration)
    row += 1

workbook.close()

In [21]:
import pandas as pd

data = pd.read_excel('file.xlsx')
data.head()

,Title,Views,Duration
0,HIKARU IS THE BULLET KING!!,"2,9K просмотров",27 минут 3 секунды
1,Hikaru Weighs In On Hans vs. Magnus!,84K просмотров,12 минут 25 секунд
2,Magnus On Hans: I'll probably win without too ...,83K просмотров,11 минут 47 секунд
3,Hans Niemann | FULL INTERVIEW,190K просмотров,32 минуты 28 секунд
4,ALIREZA SETS UP THE LAST TRAP FOR FABI,11K просмотров,12 минут 51 секунда


In [25]:
data['Views'] = data['Views'].str.replace(" просмотров","")
new = []

for i in data['Views']:
    if(i.endswith('K')):
        i = i.replace('K','')
        new.append(float(i) * 1000)
    else:
        new.append(i)

data['Views'] = new

ValueError: could not convert string to float: '2,9'